In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from torchvision.transforms import Compose, ToTensor, Resize
from sklearn.model_selection import train_test_split
import numpy as np
from numpy import asarray
from PIL import Image
import pathlib
import os
import time
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, f1_score
from torchvision.transforms import Resize, CenterCrop, ToTensor, Normalize
from google.colab import drive
import pathlib


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# device = "mps" if torch.backends.mps.is_available() else "cpu"

In [ ]:
# Hyper-parameters
num_epochs = 20
batch_size = 32
learning_rate = 0.01

#Model Parameters
num_features = 20

In [ ]:
drive.mount("/content/drive", force_remount=True)
dataset_url='/content/drive/MyDrive/plant_leaf_dataset/dataset1/'

Mounted at /content/drive


In [ ]:
# dataset = ImageFolder("/content/drive/MyDrive/plant_leaf_dataset/dataset2", transform=Compose([ToTensor(), Resize((229, 229)),CenterCrop(299),oTensor(),Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ]))
# dataset = ImageFolder(dataset_url, transform=Compose(
#     [ToTensor(),
#      Resize((224, 224)),
#      Normalize(mean = [0.485, 0.456, 0.406], std =  [0.229, 0.224, 0.225])
#     ]))
dataset = ImageFolder(dataset_url, transform=Compose([Resize((229, 229)),CenterCrop(299),ToTensor(),Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ]))
# dataset = torch.utils.data.random_split(dataset, [100, len(dataset) - 100])
dataset, testDataset=torch.utils.data.random_split(dataset, [int(0.9 * len(dataset)), len(dataset) - (int(0.9 * len(dataset)))])
trainDataset, validationDataset = torch.utils.data.random_split(dataset, [int(0.9 * len(dataset)), len(dataset) - (int(0.9 * len(dataset)))])

trainingDataLoader = DataLoader(
    trainDataset,
    batch_size=batch_size,
    shuffle=True
)

validationDataLoader = DataLoader(
    validationDataset,
    batch_size=batch_size,
    shuffle=False
)

testingDataLoader = DataLoader(
    testDataset,
    batch_size=batch_size,
    shuffle=False
)

print(len(dataset))
print(len(trainingDataLoader))
print(len(validationDataLoader))
print(len(testingDataLoader))

#Write code to display the images in each dataloader

18936
533
60
66


In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_features)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print(model)

In [ ]:
print(device)

cpu


In [ ]:
total_steps = len(trainingDataLoader)

batchwiseTrainAccuracyLog = []
batchwiseTrainLossLog = []

epochwiseTrainAccuracyLog = []
epochwiseTrainLossLog = []

batchwiseValAccuracyLog = []
batchwiseValLossLog = []

epochwiseValAccuracyLog = []
epochwiseValLossLog = []

for epoch in range(num_epochs):

    total_label_count = 0

    accumulated_train_loss=0
    accumulated_validation_loss=0

    accumulated_train_accuracy=0
    accumulated_validation_accuracy=0

    inner_loop_counter_train=0
    model.train()
    for i, data in enumerate(trainingDataLoader):
        inner_loop_counter_train+=1

        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        outputs = outputs.logits
        loss = criterion(outputs, labels)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        total_label_count += labels.size(0)
        accumulated_train_accuracy += (predicted == labels).sum().item()
        accumulated_train_loss+=loss.item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.7f}, Accuracy: {:.4f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            batchwiseTrainAccuracyLog.append((correct / total) * 100)
            batchwiseTrainLossLog.append(loss.item())
    training_accuracy = correct / total
    epochwiseTrainAccuracyLog.append(training_accuracy)
    print(f'EPOCH {epoch + 1}.....................')
    print('Accuracy on training set: {}%'.format(100 * training_accuracy))
    epochwiseTrainLossLog.append(accumulated_train_loss/inner_loop_counter_train)

    #Evaluation using validation data

    total_label_count = 0

    inner_loop_counter_validation=0

    model.eval()
    with torch.no_grad():
      for i, data in enumerate(trainingDataLoader):
        inner_loop_counter_validation+=1

        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        accumulated_validation_accuracy += (predicted == labels).sum().item()
        total_label_count += labels.size(0)
        accumulated_validation_loss+=loss.item()
        if (i + 1) % 10 == 0:
            batchwiseValAccuracyLog.append((correct / total) * 100)
            batchwiseValLossLog.append(loss.item())
    validation_accuracy = accumulated_validation_accuracy / total_label_count
    epochwiseValAccuracyLog.append(validation_accuracy)
    print('Accuracy on validation set: {}%'.format(100 * validation_accuracy))
    epochwiseValLossLog.append(accumulated_validation_loss/inner_loop_counter_validation)
print("Training Finished!")


Epoch [1/20], Step [10/533], Loss: 3.0790918, Accuracy: 6.2500%
Epoch [1/20], Step [20/533], Loss: 3.0715976, Accuracy: 9.3750%
Epoch [1/20], Step [30/533], Loss: 2.9172630, Accuracy: 9.3750%
Epoch [1/20], Step [40/533], Loss: 2.9525337, Accuracy: 0.0000%
Epoch [1/20], Step [50/533], Loss: 2.9865489, Accuracy: 9.3750%
Epoch [1/20], Step [60/533], Loss: 3.0287452, Accuracy: 6.2500%
Epoch [1/20], Step [70/533], Loss: 2.8601272, Accuracy: 3.1250%
Epoch [1/20], Step [80/533], Loss: 2.9672203, Accuracy: 12.5000%
Epoch [1/20], Step [90/533], Loss: 2.9561169, Accuracy: 18.7500%
Epoch [1/20], Step [100/533], Loss: 2.9874160, Accuracy: 15.6250%
Epoch [1/20], Step [110/533], Loss: 2.8429122, Accuracy: 6.2500%
Epoch [1/20], Step [120/533], Loss: 2.8831186, Accuracy: 6.2500%
Epoch [1/20], Step [130/533], Loss: 2.8843541, Accuracy: 9.3750%
Epoch [1/20], Step [140/533], Loss: 3.0735531, Accuracy: 18.7500%
Epoch [1/20], Step [150/533], Loss: 2.9300354, Accuracy: 12.5000%
Epoch [1/20], Step [160/533],

In [ ]:
fig,ax=plt.subplots()
epochs_list = list(range(0,20))
ax.plot(epochs_list,epochwiseTrainAccuracyLog,'g',label='Training Accuracy')
ax.plot(epochs_list,epochwiseValAccuracyLog,'b',label='Validation Accuracy')
ax.plot()
ax.set_title('GoogLeNet Accuracy Comparisions on dataset 1 with lr=0.01')
ax.set_xlabel('Number of epochs')
ax.set_ylabel('Accuracy')
ax.legend()
plt.show()

In [ ]:
fig,ax=plt.subplots()
ax.plot(epochs_list,epochwiseTrainLossLog,'g',label='Training Loss')
ax.plot(epochs_list,epochwiseValLossLog,'b',label='Validation Loss')
ax.plot()
ax.set_title('GoogLeNet Loss Comparisions on dataset 1 with lr=0.01')
ax.set_xlabel('Number of epochs')
ax.set_ylabel('Loss')
ax.legend()
plt.show()

In [ ]:
y_pred = []
groundtruth = []

model.eval()

testAccuracyLog = []
testLossLog = []

with torch.no_grad():

    inner_loop_counter_test = 0

    accumulated_test_accuracy=0
    accumulated_test_loss=0

    for images, labels in testingDataLoader:
        images = images.to(device)
        labels = labels.to(device)
        inner_loop_counter_test+=1
        test_loss=0
        outputs = model(images)
        test_loss=criterion(outputs,labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_pred.append(predicted)
        groundtruth.append(labels)
        accumulated_test_loss+=test_loss.item()
    testLossLog.append(accumulated_test_loss/inner_loop_counter_test)
    test_accuracy=correct/total
    print('Accuracy on test set: {}%'.format(100 * test_accuracy))
    testAccuracyLog.append(test_accuracy)

predictions_np = [tensor.detach().cpu().numpy() for tensor in y_pred]
ground_truth_np = [tensor.detach().cpu().numpy() for tensor in groundtruth]

predictions_np = np.concatenate(predictions_np)
ground_truth_np = np.concatenate(ground_truth_np)

accuracy = accuracy_score(ground_truth_np, predictions_np)
recall = recall_score(ground_truth_np, predictions_np, average='weighted')
conf_matrix = confusion_matrix(ground_truth_np, predictions_np)
f_score = f1_score(ground_truth_np, predictions_np, average='weighted')

print("Dataset 1 - Test Metrics\nAccuracy:", accuracy*100)
print("Recall:", recall*100)
print("F-score:", f_score*100)

print("Confusion Matrix:")

plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix,annot=True,fmt='d',cmap='Blues',cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix GoogLeNet on Dataset 1 lr=0.01')
plt.show()